## Init and Run GMM

In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
#this sets the backend to jupyter/ipython that (i think) displays
#     images directly. anyway, it prevents the matplotlib framework
#     python error that is my least favorite thing eeeevvvveeeer.
%matplotlib notebook

import sys
sys.path.append("..")

import gmix_model as gmix
import numpy as np
import tdb as tdb
import tensorflow as tf
import matplotlib.pyplot as plt

import os
os.chdir('/Users/azane/GitRepo/spider')

In [3]:
#read in training and test data
s_x, s_t = gmix.get_xt_from_npz('data/toy_train.npz')
t_x, t_t = gmix.get_xt_from_npz('data/toy_train.npz')

In [4]:
#create gmm with data
gmm = gmix.GaussianMixtureModel(s_x, s_t, t_x, t_t,
                               numGaussianComponents=3, hiddenLayerSize=10,
                               learningRate=0.004)

In [5]:
%%capture
#d will be a dictionary of evaluated tensors under their standard name.
runTimes = 10000
reports = 20

d = gmm.train(iterations=runTimes, testBatchSize=1000,
          trainBatchSize=500, reportEvery=int(runTimes/reports))


## Debugging

In [6]:
print d['tot_likelihood']

[ 1.48431003  2.30454278  2.39593673  1.66984272  0.65761882  2.04749799
  0.55651861  4.6108942   1.68620598  2.25969934  1.02339613  2.59868336
  1.6833204   1.91930485  2.85912204  4.06610441  1.5611912   1.74036241
  0.91463971  0.82067323  4.68973112  2.3794415   1.34507573  1.73283637
  2.301337    1.3646698   4.98818159  4.56317186  2.29550743  3.75775766
  0.10459018  1.65616775  2.21044445  4.56099844  3.3331871   3.80367041
  3.68708301  1.9367739   0.8485809   1.41132712  4.96725416  0.61761588
  2.12481737  1.12779844  2.32161808  2.62537575  0.46044651  3.14516783
  0.47936672  2.7870307   1.48949778  3.66664577  1.93404102  1.09699368
  1.26229513  1.85049343  4.75915003  4.38872242  1.69223857  1.63180935
  3.75865197  3.31860709  1.50978863  2.70437527  1.28546739  3.69838953
  1.74098933  1.42518866  1.57427144  1.5850935   4.61447954  1.68499017
  2.49420476  2.78126717  0.9335984   2.12017012  0.70193112  4.25545835
  0.98617625  3.98937798  1.53364027  1.29968929  1

In [7]:
#%%capture
print 'grad_w1'
print d['grad_w1']
print 'grad_b1'
print d['grad_b1']
print 'grad_w2'
print d['grad_w2']
print 'grad_b2'
print d['grad_b2']
print 'grad_w3'
print d['grad_w3']
print 'grad_b3'
print d['grad_b3']

grad_w1
[[  5.62771797  80.09931946 -27.33720589 -21.19077492  33.47651291
   36.2424202  -10.43572235 -22.67038155 -45.25403976 -53.45733643]]
grad_b1
[-298.81866455  -28.99754333 -160.3789978    35.33131409 -114.64504242
   79.58921814  203.29785156   55.02895737  -32.18833923  -47.80590057]
grad_w2
[[  9.98393631   3.78816414 -35.55271149  -6.49385309  -8.14875889
   -2.58450222   5.68263197   5.52193642 -29.90649223  20.22596931]
 [-23.63088417 -11.22378731  78.13067627  15.69320393  25.03497124
   12.47793579 -22.63183784  -9.55712414  55.498703   -30.4292202 ]
 [-27.34448624 -14.42572975  84.02080536  15.57085705  34.98750305
   19.74419594 -28.28468513  -9.58355141  51.4324913  -20.34169769]
 [-14.66982079  15.28945827  49.68463135   3.16592288   4.40371275
  -13.08437347  28.58161163  -9.41571903  72.01451874 -62.51326752]
 [-10.79598236  -1.29749095  42.53688812   4.92924166   5.0562191
   -2.58982754   5.29484797  -9.67914009  45.32346725 -36.39640427]
 [  1.28574383   2.1942

In [8]:
#%%capture
print 'calc_agg_grad_w1'
print d['calc_agg_grad_w1']
print 'calc_agg_grad_b1'
print d['calc_agg_grad_b1']
print 'calc_agg_grad_w2'
print d['calc_agg_grad_w2']
print 'calc_agg_grad_b2'
print d['calc_agg_grad_b2']
print 'calc_agg_grad_w3'
print d['calc_agg_grad_w3']
print 'calc_agg_grad_b3'
print d['calc_agg_grad_b3']

calc_agg_grad_w1
[[  1.47641094e-05   3.12796299e-04   8.58000494e-05  -2.87728006e-04
    1.98091235e-04   1.33458176e-04   2.83207373e-05  -9.95104274e-05
   -2.35662970e-04   1.30299988e-04]]
calc_agg_grad_b1
[-0.00123517 -0.0001689  -0.00081334  0.00015824 -0.00043762  0.00034058
  0.00093392  0.00023926 -0.00019115 -0.00032166]
calc_agg_grad_w2
[[  4.67123245e-05   1.69229588e-05  -2.53236067e-04  -7.71087289e-05
   -3.91410213e-05  -1.00510195e-04   1.19256481e-04   2.27583860e-05
   -1.51052518e-04   9.63869461e-05]
 [ -1.08793261e-04  -5.07625155e-05   5.03362040e-04   1.29644657e-04
    1.12442438e-04   2.58126849e-04  -1.90396051e-04  -3.73674338e-05
    2.69242271e-04  -1.46967315e-04]
 [ -1.25479128e-04  -6.53906391e-05   5.15287393e-04   1.04155770e-04
    1.52445195e-04   3.24988272e-04  -1.38956981e-04  -3.61208440e-05
    2.43357572e-04  -1.01649763e-04]
 [ -7.50885665e-05   7.47959639e-05   5.38999215e-04   2.21338327e-04
    4.11743276e-05   3.62207866e-05  -3.7734553

In [9]:
inSize = 300.
ins = np.expand_dims((np.arange(inSize)/inSize), 1) #get input 0-1

In [10]:
lRate = 1

np_grad1 =  np.tanh( np.dot(ins, d['calc_agg_grad_w1']*lRate)\
                    + d['calc_agg_grad_b1']*lRate)
np_grad2 =  np.tanh( np.dot(np_grad1, d['calc_agg_grad_w2']*lRate)\
                    + d['calc_agg_grad_b2']*lRate)
np_grad3 =  np.tanh( np.dot(np_grad2, d['calc_agg_grad_w3']*lRate)\
                    + d['calc_agg_grad_b3']*lRate)

#np_grad1 =  np.dot(grad_ins, d['grad_w1']*lRate)# + np.tanh(d['grad_b1']*lRate)
#np_grad2 =  np.dot(np_grad1, d['grad_w2']*lRate)# + np.tanh(d['grad_b2']*lRate)
#np_grad3 =  np.dot(np_grad2, d['grad_w3']*lRate)# + np.tanh(d['grad_b3']*lRate)

#print np_grad3
#plt.plot(ins, np_grad3)

In [11]:
s = 200  # sample index for testing

In [12]:
%%capture
print 'netIn:'
print d['netIn'][s]

small_change = d['netIn'][s] + 0.05
print 'small_change:'
print small_change


In [13]:
%%capture
print 'w1:'
print d['w1']
print 'b1:'
print d['b1']
print 'lay1:'
print d['lay1'][s]


np_calc1 =  np.tanh( np.dot(d['netIn'][s], d['w1']) + d['b1'])
np_dif1 =  np_calc1 - d['lay1'][s]

print 'numpy calc:'
print np_calc1
print 'numpy calc dif:'
print np_dif1

print '------------------------------'
np_calc_sc1 = np.tanh( np.dot(small_change, d['w1']) + d['b1'])
np_dif_sc1 = np_calc1 - np_calc_sc1

print 'numpy calc:'
print np_calc1
print 'numpy calc of small change:'
print np_calc_sc1
print 'numpy calc dif with small change:'
print np_dif_sc1
print '------------------------------'


In [14]:
%%capture
print 'w2:'
print d['w2']
print 'b2:'
print d['b2']
print 'lay2:'
print d['lay2'][s]


np_calc2 =  np.tanh( np.dot(np_calc1, d['w2']) + d['b2'])
np_dif2 =  np_calc2 - d['lay2'][s]

print 'numpy calc:'
print np_calc2
print 'numpy calc dif:'
print np_dif2


In [15]:
%%capture
print 'w3:'
print d['w3']
print 'b3:'
print d['b3']
print 'netOut:'
print d['netOut'][s]


np_calc3 =  np.tanh( np.dot(np_calc2, d['w3']) + d['b3'])
np_dif3 =  np_calc3 - d['netOut'][s]

print 'numpy calc:'
print np_calc3
print 'numpy calc dif:'
print np_dif3